# Oxylabs Search


This notebook goes over how to use the Oxylabs search component.

First, you need to set up the proper API keys and environment variables. Create your API user credentials: Sign up for a free trial or purchase the product in the [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration) to create your API user credentials (OXYLABS_USERNAME and OXYLABS_PASSWORD).



## Setup
The integration lives in the `langchain-community` package.

In [1]:
# %pip install -U langchain-community oxylabs

In [2]:
import getpass
import os
from pprint import pprint

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

## Basic usage

In [3]:
from langchain_community.utilities import OxylabsSearchAPIWrapper

search = OxylabsSearchAPIWrapper()

In [4]:
pprint(search.run("Python Programming Language"))

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '    IMAGES-ITEM-4: Redacted base64 image string...\n'
 '    IMAGES-ITEM-5: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?num=5&sca_esv=808c4f249ba0e78b&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwiD79n0xb2JAxXHd2wGHY51MFYQmxMoAHoECDEQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4

## Number of results
You may use the parameters `start_page`, `pages` and `limit` to specify the number of results. The default values are:  
    * `start_page = 1`  
    * `pages = 1`  
    * `limit = 5`  

In [5]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 2,
        "limit": 10
    }
)


In [ ]:
pprint(search.run("Python Programming Language"))

## Result Filtering / Ordering

### Default result group order :  
* `knowledge_graph`  
* `combined_search_result`  
* `product_information`  
* `local_information`  
* `search_information`  
  
You may want to adjust the display order of result groups and exclude the `product_information`, `local_information`  and `combined_search_result` groups in cases where only abstract knowledge is needed. For example, this configuration allows tailored result presentation.

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 1,
        "limit": 10,
        "geo_location": "Kairo,Egypt",
        "result_categories": [
            "search_information",
            "knowledge_graph",            
        ]
    }
)
pprint(search.run("Visiting Great Pyramid of Giza in Egypt"), indent=4)


You may want to adjust the display order of result groups and exclude abstract knowledge when focusing specifically on products and local services. In the following example reorders result groups to prioritise search information and local services, omitting abstract knowledge to enhance relevance for this context.

In [ ]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 15,
        "geo_location": "Belgium",
        "result_categories": [
            "search_information",
            "local_information",            
        ]
    }
)
pprint(search.run("Open Working Space in Belgium."))

## Raw Results

In [ ]:
search = OxylabsSearchAPIWrapper()

In [ ]:
search.results("apples")

## Tool Usage

In [ ]:
import os

from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()


In [ ]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)
tool_results

In [ ]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)
tool_run

In [ ]:
import json

# .invoke wraps utility.results
response_results = tool_results.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "China",  
})
response_results = json.loads(response_results)
for item in response_results:
    print(item)

In [ ]:
# .invoke wraps utility.results
response_run = tool_run.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "China",    
})

pprint(response_run)

# Chaining

In [ ]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community

In [ ]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

In [ ]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an politological analysis essay from search results."})

In [ ]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})